In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# load stitchnet
# restitching
# evaluate accuracy vs # of samples used to stitch

In [3]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset, load_dl

from tqdm import tqdm
import torch
import numpy as np
import random

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
fragmentsMap = {}
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    js = []
    for j,fragmentFile in enumerate(fragmentFiles):
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
        js.append(j)
    net1 = Net(onnxFragments, i)
    for j in js:
        fragmentsMap[(i,j)] = net1[j]

In [5]:
import pandas as pd
data = pd.read_csv('_results/result_paper.txt', names=['score','acc','macs','params','path','fragments'])

In [ ]:
from ast import literal_eval as make_tuple
from stitchnet.stitchonnx.utils import load_onnx_model
from stitchnet.stitchonnx.utils import stitch_fragments
from stitchnet.stitchonnx.utils import accuracy_score_net
import os

STITCH_BATCH_SIZES = [
    2,4,8,32, 64, 128, 256
]
RESULT_NAME = 'stitch_samples5'

os.makedirs(f'_results/{RESULT_NAME}', exist_ok=True)
for netId,fragments in enumerate(data['fragments']):
    with open(f'_results/{RESULT_NAME}/{netId:03}.txt', 'w') as file:
        for batch_size in STITCH_BATCH_SIZES:
            # random.seed(51)
            # np.random.seed(24)
            # torch.manual_seed(77)

            dl_score = load_dl(dataset_train, batch_size)
            data_score,t = next(iter(dl_score))
            data_score = data_score.numpy()

            fs = make_tuple(fragments)
            onnx_fragments = []
            for f in fs:
                netId,fId = f
                onnx_fragments.append(fragmentsMap[(netId,fId)])

            # stitching fragments
            fragmentC = onnx_fragments[0]
            for k in range(1,len(onnx_fragments)):
                onnx_model = stitch_fragments(fragmentC, onnx_fragments[k], data_score)
                net = Net([onnx_model])
                fragmentC = net[0]

            acc = accuracy_score_net(net, dataset_val, bs=128)
            print(batch_size,acc)
            file.write(f'{batch_size},{acc}\n')
            file.flush()

/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


epoch 0 loss 2.40585208819449e-09
epoch 1 loss 1.2334950935989902e-09
epoch 2 loss 3.2165211143866547e-10
epoch 3 loss 7.226906188253702e-10
epoch 4 loss 1.5162736361409411e-09
epoch 5 loss 1.4117943406316153e-09
epoch 6 loss 6.811812775174733e-10
epoch 7 loss 1.4067046323699963e-10
epoch 8 loss 4.016491257860175e-10
epoch 9 loss 8.729581069594472e-10
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0


100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


2 0.45054945054945056
epoch 0 loss 7.168842078415849e-10
epoch 1 loss 2.537603378456239e-10
epoch 2 loss 2.2303676722918214e-10
epoch 3 loss 3.9665550161615584e-10
epoch 4 loss 2.5576307418544615e-10
epoch 5 loss 7.538674455301666e-11
epoch 6 loss 1.2200142557537878e-10
epoch 7 loss 1.764996927760629e-10
epoch 8 loss 9.692939637942774e-11
epoch 9 loss 4.602207960860564e-11
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss

100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


...

128 0.7912087912087912
epoch 0 loss 5.1379715685492726e-08
epoch 1 loss 2.1787645849842386e-09
epoch 2 loss 1.1482434972547871e-09
epoch 3 loss 9.573724816684089e-10
epoch 4 loss 8.967111069553769e-10
epoch 5 loss 8.698786335865032e-10
epoch 6 loss 8.533909715897074e-10
epoch 7 loss 8.431256454451479e-10
epoch 8 loss 8.34738668806507e-10
epoch 9 loss 8.250355392105209e-10
epoch 0 loss 1.287011492959286e-08
epoch 1 loss 3.4742602598782654e-09
epoch 2 loss 3.3013477813879615e-09
epoch 3 loss 3.2930134042145264e-09
epoch 4 loss 3.2896213468660795e-09
epoch 5 loss 3.2938231505992372e-09
epoch 6 loss 3.293006201030662e-09
epoch 7 loss 3.2893469337977766e-09
epoch 8 loss 3.2887544456258495e-09
epoch 9 loss 3.2946730155621805e-09
epoch 0 loss 2.0030975520866186e-08
epoch 1 loss 1.457473540207403e-08
epoch 2 loss 1.435556502911184e-08
epoch 3 loss 1.4330448584477804e-08
epoch 4 loss 1.4329973606201598e-08
epoch 5 loss 1.4330643851110028e-08
epoch 6 loss 1.4321380700613593e-08
epoch 7 loss 1.43

100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


256 0.7142857142857143
epoch 0 loss 1.1321124778381393e-09
epoch 1 loss 7.01362328677801e-10
epoch 2 loss 2.4662363296210995e-10
epoch 3 loss 2.637560756602834e-10
epoch 4 loss 5.559099225759499e-10
epoch 5 loss 6.715774688378982e-10
epoch 6 loss 5.135391968398563e-10
epoch 7 loss 2.127331616762087e-10
epoch 8 loss 8.59621229556044e-11
epoch 9 loss 1.9624543173248777e-10
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


2 0.4835164835164835
epoch 0 loss 1.534841940480558e-09
epoch 1 loss 3.0542403661813583e-10
epoch 2 loss 9.174796816198981e-10
epoch 3 loss 8.019110850130037e-10
epoch 4 loss 1.709978794745168e-10
epoch 5 loss 5.30311046409742e-10
epoch 6 loss 4.008229468497491e-10
epoch 7 loss 1.0290075025870805e-10
epoch 8 loss 3.192179226332581e-10
epoch 9 loss 2.1192648611462142e-10
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0


In [7]:
# onnx_fragments

In [4]:
# with open('_results/result_paper.txt', 'r') as f:
#     for line in f:
#         print(line)

In [28]:
# onnx_fragments[0].graph.input